In [4]:
%load_ext autoreload
%autoreload 2


In [3]:
import pandas as pd

# importa desde tu paquete (el nombre debe coincidir con tu __init__.py)
from Isea import ScatterBrush   # si tu paquete se llama distinto, ajusta aquí

# lee tu CSV local
df = pd.read_csv("Isea/datos_vis.csv")

# lanza el scatter con brush (área)
ScatterBrush(df, x="horsepower", y="weight", color="origin")


<IPython.core.display.Javascript object>

In [5]:
from IPython.display import HTML, Javascript, display
display(HTML("<div id='ok' style='padding:6px;border:1px solid #ddd'>Esperando JS…</div>"))
display(Javascript("document.getElementById('ok').innerText = 'JS OK ✅';"))


<IPython.core.display.Javascript object>

In [6]:
from IPython.display import Javascript, display
display(Javascript("""
(async () => {
  const mod = await import('https://cdn.jsdelivr.net/npm/d3@7/+esm');
  const d3v = mod.default ?? mod;
  document.getElementById('ok').innerText = 'D3 cargado ✅ (keys: ' + Object.keys(d3v).length + ')';
})();
"""))


<IPython.core.display.Javascript object>